In [36]:
using Printf
Base.show(io::IO, f::Float64) = @printf(io, "%.3e", f)
using LinearAlgebra
using SparseArrays
using Random

using BenchmarkTools
using Base.Threads

In [7]:
#=AJUSTE DE NUMERO DE HILOS 

nthreads = 2, 4, 6, 8, 16, 32


export JULIA_NUM_THREADS=4   # Linux  
set JULIA_NUM_THREADS=4      # CMD Windows  

$ julia --threads 4
julia> Threads.nthreads()
julia> Threads.threadid()

=#


Threads.nthreads()      #numero de hilos
Threads.threadid()

1

### Ejemplos de sistemas de ecuaciones
### Casos diagonal dominante para una convergencia mas rapida

In [8]:
# sistema de ecuaciones Ax=b 3x3
# CASO DIAGONAL DOMINANTE PARA UNA RAPIDA CONVERGENCIA
A = [3 -2 0; 1 3 -1; 2 1 4]
b = [-3, -3, 6]
A\b

3-element Vector{Float64}:
 -1.000e+00
  0.000e+00
  2.000e+00

In [ ]:
# sistema de ecuaciones Ax=b 3x3

# CASO DIAGONAL DOMINANTE PARA UNA RAPIDA CONVERGENCIA
#=cada elemento de la diagonal es mayor que el modulo de la suma de los otros elementos 
  en este renglon

  el valor absoluto del termino sobre la diagonal es mayor que o igual a la suma de valores absolutos del resto
  de los terminos en ese renglon
=#

A = [5 2 1; 0 -3 2; -4 1 6]
b = [1, 0, -1]
A\b

In [ ]:
#sistema de ecuaciones 4x4
# CASO DIAGONAL DOMINANTE PARA UNA RAPIDA CONVERGENCIA
A = [ 10  2  -1  2;
       1  5   1  0;
       1 -2  -5  1;
       3  0   0  9 ]

b = [-4, 1, 2, 10]
#@time A\b
A

In [9]:
# revisa si una matriz es diagonalmente dominante 
all(sum(abs.(A),dims=2) .<= 2abs.(diag(A)))

true

### Generador de matrices aleatorias

In [ ]:
#generador de matrices aleatorias 1

#=DATOS PARA LAS PRUEBAS

dimensiones 20, 40, 60, 80, 100
=#


print("Dar la dimension del sistema de ecuaciones: ")
line = readline(stdin)
dim = parse(Int, line)
Random.seed!(123);
mat = rand(dim, dim)
# ------------------------------------------------
#mat = sprand(dim,dim,2/5)
#mat=Matrix(mat)
# -----------------------------------------
#matriz dispersa
brow = randperm!(MersenneTwister(1234), Vector{Int}(undef, dim))
bcol = randperm!(MersenneTwister(1234), Vector{Int}(undef, dim))
data = randperm!(MersenneTwister(1234), Vector{Int}(undef, dim))

sparseA = sparse(brow, bcol, data, dim, dim)    
#definicion de matriz dispersa A
sparseA =Matrix(sparseA)
mat = float(sparseA)
mat = trunc.(Int,mat)   #regresa el valor entero mas proximo del mismo tipo a 'mat'


# revisa si una matriz es diagonalmente dominante 
all(sum(abs.(mat),dims=2) .<= 2abs.(diag(mat)))


# -----------------------------------------------------------------
# paso de parametros por el generador de matrices aleatorias 1

A = mat
b = bcol
# ------------------------------------------------------------------

### Calculo de las matrices auxiliares

In [10]:
# prueba para el algoritmo MCMC
inv(A)

3×3 Matrix{Float64}:
  2.549e-01   1.569e-01  3.922e-02
 -1.176e-01   2.353e-01  5.882e-02
 -9.804e-02  -1.373e-01  2.157e-01

In [14]:
#calculo de las matrices auxilliares
function calculosauxialiares(A, b)
    
    
    global nA = norm(A)
    
    global M = diagm(0 => diag(A))
    
    global N = M-A   #obtener el numero de cadenas de Markov
    
    global T = inv(M) * N
    
    global f = inv(M) * b    
end

calculosauxialiares (generic function with 1 method)

In [15]:
calculosauxialiares(A, b)

3-element Vector{Float64}:
 -1.000e+00
 -1.000e+00
  1.500e+00

### Algoritmo MCMC para el cálculo del vector solución

In [16]:
T

3×3 Matrix{Float64}:
  0.000e+00   6.667e-01  0.000e+00
 -3.333e-01   0.000e+00  3.333e-01
 -5.000e-01  -2.500e-01  0.000e+00

In [17]:
f

3-element Vector{Float64}:
 -1.000e+00
 -1.000e+00
  1.500e+00

In [19]:
# paso de parametros
nT, mT = size(T)

(3, 3)

### matriz de probabilidad 𝑃 y el vector inicial de probabilidad 𝑝

In [20]:
S = fill(0, nT);
[S[i] += 1 for i in 1:nT, j in 1:mT if T[i,j] != 0];
S

3-element Vector{Int64}:
 1
 2
 2

In [21]:
P = fill(0., nT, mT)
[P[i,j]= 1/S[i] for i in 1:nT, j in 1:mT if T[i,j] != 0 ]
P

3×3 Matrix{Float64}:
 0.000e+00  1.000e+00  0.000e+00
 5.000e-01  0.000e+00  5.000e-01
 5.000e-01  5.000e-01  0.000e+00

In [22]:
Pi = [1/nT for i in 1:nT]

3-element Vector{Float64}:
 3.333e-01
 3.333e-01
 3.333e-01

Definimos el error de exactitud 𝜖 y el error estadístico 𝛿 

In [23]:
# valores de epsilon para pruebas
# 0.05 0.06 0.07 0.08 0.09 0.10 

#=EXPERIMENTO 1

epsilon = 0.1
delta = 5
=#

ϵ = 0.01  
δ = 0.1 

1.000e-01

El número  𝑁  de cadenas de Markov

In [24]:
N = floor((0.6745/δ)^2*((norm(f)^2)/(1-norm(T))^2)) + 1

#valor de prueba para el numero N de cadenas de Markov
# N = 100


# Otros enfoques consideran la cota inferior:
#N = floor((0.6745/δ)^2*(1/(1-norm(T))^2))

1.763e+06

Algoritmo MCMC serial

In [25]:
function mcmcserial(ϵ, N)
    Xs = fill(0., mT)
    for i in 1:mT  # ---------inicio de region paralela  for 1
        W_0 = 1
        for s in 1:N  #for 2 
            #local W_0 = 1
            W = W_0
            k = 0
            point = i
            X = W_0 * f[i]

            while abs(W) >= ϵ
                # generar un r.v sig punto distribuido sobre el i -th
                # renglon de la matriz P como:
                nextpoint  = 1
                u = rand()
                #println(u)
                while u >= sum(P[point, 1:nextpoint])
                     nextpoint = nextpoint + 1
                    #println(nextpoint)
                end
                k = k + 1
                if T[point, nextpoint] != 0 

                    W_new = W *(T[point, nextpoint]/P[point, nextpoint])
                    X = X + W_new * f[nextpoint]

                end
                point = nextpoint
                W = W_new
            end  # fin de while 
        Xs[i] += X
        end              # fin for 2
    end                 # ------------ fin de region paralela fin for 1 
    Xs = Xs/N
end

mcmcserial (generic function with 1 method)

In [26]:
A

3×3 Matrix{Int64}:
 3  -2   0
 1   3  -1
 2   1   4

In [27]:
b 

3-element Vector{Int64}:
 -3
 -3
  6

In [37]:
Xs1 = @btime mcmcserial(ϵ, N)

  35.510 s (1158509870 allocations: 25.37 GiB)


3-element Vector{Float64}:
 -9.954e-01
  2.657e-03
  1.997e+00

In [30]:
norm(b-A*Xs1)

1.490e-02

Algoritmo MCMC paralelo

In [31]:
function mcmcparalelo(ϵ, N)
    Xs = fill(0., mT)
    Threads.@threads for i in 1:mT  # ---------inicio de region paralela  for 1
        W_0 = 1
        for s in 1:N  #for 2 
            #local W_0 = 1
            local W = W_0
            local k = 0
            local point = i
            local X = W_0 * f[i]

            while abs(W) >= ϵ
                # generar un r.v sig punto distribuido sobre el i -th
                # renglon de la matriz P como:
                nextpoint  = 1
                u = rand()
                #println(u)
                while u >= sum(P[point, 1:nextpoint])
                     nextpoint = nextpoint + 1
                    #println(nextpoint)
                end
                k = k + 1
                if T[point, nextpoint] != 0 
                    #=desactivar el chequeo de limites con @inbounds 
                      reduce el tiempo de ejecucion
                    =#                       
                    @inbounds W_new = W *(T[point, nextpoint]/P[point, nextpoint])
                    @inbounds X = X + W_new * f[nextpoint]
                end
                point = nextpoint
                W = W_new
            end  # fin de while 
        Xs[i] += X
        end              # fin for 2
    end                 # ------------ fin de region paralela fin for 1 
    Xs = Xs/N
end

mcmcparalelo (generic function with 1 method)

In [32]:
A

3×3 Matrix{Int64}:
 3  -2   0
 1   3  -1
 2   1   4

In [33]:
b

3-element Vector{Int64}:
 -3
 -3
  6

In [34]:
Xs2 = @btime mcmcparalelo(ϵ, N)

LoadError: LoadError: UndefVarError: @btime not defined
in expression starting at In[34]:1

In [ ]:
norm(b-A*Xs2)